In [1]:
import torch
from model.sentiment_model import Cls

Some weights of the model checkpoint at klue/roberta-small were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-small and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it f

### model load

In [2]:
model = Cls()
state_dict = torch.load('/home/sangyeon/sentiment/sentiment_state_2.pt', map_location='cpu') 
# dataparallel로 학습했기에 module이 생겨서 새로운 dictionary생성
new_state_dict = {}
for k, v in state_dict.items():
    name = k[7:]
    new_state_dict[name] = v
model.load_state_dict(new_state_dict)

<All keys matched successfully>

### tokenizer

In [3]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-small")
n_added_token=tokenizer.add_special_tokens({"additional_special_tokens":['[대분류]','[소분류]']})
tokenizer.model_max_length = 100

In [4]:
input_ids , attention_mask =  tokenizer(
    '이번 프로젝트에서 발표를 하는데 내가 실수하는 바람에 우리 팀이 감점을 받았어. 너무 미안해.'+'[SEP]'+'내 능력이 부족한 거 같은데 그만 다녀야 될거같아.'+' 감정 대분류 : [대분류] - 감정 소분류 : [소분류]',
    padding='max_length',
    return_tensors = 'pt',
    return_token_type_ids=False,
    add_special_tokens=False # [ClS]와 마지막 [SEP]은 사용하지 않을 것이기에 제외
    ).values() 
idx1 = torch.argwhere(input_ids==32000)[0,1] # 대분류의 index
idx2 = torch.argwhere(input_ids==32001)[0,1] # 소분류의 index

'''
입력 문장
이번 프로젝트에서 발표를 하는데 내가 실수하는 바람에 우리 팀이 감점을 받았어. 너무 미안해.
[SEP]
내 능력이 부족한 거 같은데 그만 다녀야 될거같아.
'''

'\n입력 문장\n이번 프로젝트에서 발표를 하는데 내가 실수하는 바람에 우리 팀이 감점을 받았어. 너무 미안해.\n[SEP]\n내 능력이 부족한 거 같은데 그만 다녀야 될거같아.\n'

### sentiment

In [5]:
sentiment1 = {
    '분노': 0, 
    '기쁨': 1, 
    '불안': 2,
    '당황': 3, 
    '슬픔': 4, 
    '상처': 5
}
sentiment2 = {
    '노여워하는': 0,
    '느긋': 1,
    '걱정스러운': 2,
    '당혹스러운': 3,
    '당황': 4,
    '마비된': 5,
    '만족스러운': 6,
    '배신당한': 7,
    '버려진': 8,
    '부끄러운': 9,
    '분노': 10,
    '불안': 11,
    '비통한': 12,
    '상처': 13,
    '성가신': 14,
    '스트레스 받는': 15,
    '슬픔': 16,
    '신뢰하는': 17,
    '신이 난': 18,
    '실망한': 19,
    '악의적인': 20,
    '안달하는': 21,
    '안도': 22,
    '억울한': 23,
    '열등감': 24,
    '염세적인': 25,
    '외로운': 26,
    '우울한': 27,
    '고립된': 28,
    '좌절한': 29,
    '후회되는': 30,
    '혐오스러운': 31,
    '한심한': 32,
    '자신하는': 33,
    '기쁨': 34,
    '툴툴대는': 35,
    '남의 시선을 의식하는': 36,
    '회의적인': 37,
    '죄책감의': 38,
    '혼란스러운': 39,
    '초조한': 40,
    '흥분': 41,
    '충격 받은': 42,
    '취약한': 43,
    '편안한': 44,
    '방어적인': 45,
    '질투하는': 46,
    '두려운': 47,
    '눈물이 나는': 48,
    '짜증내는': 49,
    '조심스러운': 50,
    '낙담한': 51,
    '환멸을 느끼는': 52,
    '희생된': 53,
    '감사하는': 54,
    '구역질 나는': 55,
    '괴로워하는': 56,
    '가난한, 불우한': 57
}
inverse_sentiment1 = {v:k for k,v in sentiment1.items()}
inverse_sentiment2 = {v:k for k,v in sentiment2.items()}

### result


In [6]:
out1 , out2 = model(input_ids , attention_mask, idx1, idx2)

In [7]:
inverse_sentiment1[torch.argmax(out1).item()]

'불안'

In [8]:
inverse_sentiment2[torch.argmax(out2).item()]

'두려운'